# Doing all of the EDA and Model work

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch

## Load Data

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.4103, 0.3823, 0.8095],
        [0.6915, 0.2055, 0.5150],
        [0.8426, 0.7800, 0.6047],
        [0.4650, 0.5369, 0.2615],
        [0.9701, 0.6494, 0.3163]])


In [ ]:
import os

BASE_DIR = "../data/mushrooms"

for split in ["train", "val", "test"]:
    for cls in ["edible", "poisonous"]:
        p = os.path.join(BASE_DIR, split, cls)
        print(split, cls, "->", "FOUND" if os.path.exists(p) else "MISSING", "|", p)


train edible -> FOUND | ../data/mushrooms\train\edible
train poisonous -> FOUND | ../data/mushrooms\train\poisonous
val edible -> FOUND | ../data/mushrooms\val\edible
val poisonous -> FOUND | ../data/mushrooms\val\poisonous
test edible -> FOUND | ../data/mushrooms\test\edible
test poisonous -> FOUND | ../data/mushrooms\test\poisonous


In [20]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Basic transform (we'll improve this later)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

DATA_DIR = "../data/mushrooms"

train_dataset = datasets.ImageFolder(f"{DATA_DIR}/train", transform=transform)
val_dataset   = datasets.ImageFolder(f"{DATA_DIR}/val", transform=transform)
test_dataset  = datasets.ImageFolder(f"{DATA_DIR}/test", transform=transform)

print("Classes:", train_dataset.classes)
print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))


Classes: ['edible', 'poisonous']
Train size: 26886
Val size: 8851
Test size: 3831


In [23]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

val_loader = DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

test_loader = DataLoader(
    test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

print("Batches per epoch (train):", len(train_loader))
print("Batches (val):", len(val_loader))
print("Batches (test):", len(test_loader))

Batches per epoch (train): 841
Batches (val): 277
Batches (test): 120


In [18]:
images, labels = next(iter(train_loader))

print("Batch images shape:", images.shape)
print("Batch labels shape:", labels.shape)
print("Unique labels in batch:", labels.unique().tolist())

Batch images shape: torch.Size([32, 3, 224, 224])
Batch labels shape: torch.Size([32])
Unique labels in batch: [0, 1]


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

from torchvision.models import ResNet18_Weights
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Step 5 (also in roadmap): replace final layer for 2 classes
model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)
print(model.fc)

In [8]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

print("Loss:", type(criterion).__name__)
print("Optimizer:", type(optimizer).__name__, "lr =", optimizer.param_groups[0]["lr"])

Loss: CrossEntropyLoss
Optimizer: Adam lr = 0.0001


In [9]:
# Step 7A: one-batch forward pass sanity check (no training yet)

model.train()  # training mode

images, labels = next(iter(train_loader))
images = images.to(device)
labels = labels.to(device)

outputs = model(images)

print("outputs shape:", outputs.shape)  # expect [batch_size, 2]
print("labels shape:", labels.shape)    # expect [batch_size]
print("example logits (first row):", outputs[0].detach().cpu().tolist())

outputs shape: torch.Size([32, 2])
labels shape: torch.Size([32])
example logits (first row): [-0.23532402515411377, -0.7915375828742981]


In [24]:
import torch

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * images.size(0)

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc

# Run exactly 1 epoch first
train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
val_loss, val_acc = evaluate(model, val_loader, criterion, device)

print(f"Epoch 1 | train loss {train_loss:.4f} acc {train_acc:.4f} | val loss {val_loss:.4f} acc {val_acc:.4f}")

Epoch 1 | train loss 0.2036 acc 0.9134 | val loss 0.7614 acc 0.7519


In [25]:
import os
import torch

os.makedirs("checkpoints", exist_ok=True)

checkpoint_path = "checkpoints/resnet18_epoch1.pt"
torch.save({
    "model_state_dict": model.state_dict(),
    "class_to_idx": train_dataset.class_to_idx,
    "classes": train_dataset.classes
}, checkpoint_path)

print("Saved checkpoint to:", checkpoint_path)

Saved checkpoint to: checkpoints/resnet18_epoch1.pt


In [26]:
import torch

def run_validation(model, val_loader, criterion, device):
    model.eval()                 # turn off dropout/batchnorm training behavior
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():        # disable gradient tracking
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss = total_loss / total
    val_acc = correct / total

    return val_loss, val_acc

In [27]:
val_loss, val_acc = run_validation(model, val_loader, criterion, device)

print(f"Validation loss: {val_loss:.4f}")
print(f"Validation acc:  {val_acc:.4f}")

Validation loss: 0.7614
Validation acc:  0.7519
